<a href="https://colab.research.google.com/github/ah00ee/federated-learning-tutorial/blob/main/FL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This code is from https://blog.openmined.org/upgrade-to-federated-learning-in-10-lines/
!pip install syft==0.2.9

     |████████████████████████████████| 433 kB 8.8 MB/s 
     |████████████████████████████████| 57 kB 4.1 MB/s 
     |████████████████████████████████| 66 kB 4.0 MB/s 
     |████████████████████████████████| 1.8 MB 23.4 MB/s 
     |████████████████████████████████| 20.1 MB 10.4 MB/s 
     |████████████████████████████████| 2.0 MB 45.4 MB/s 
     |████████████████████████████████| 200 kB 51.5 MB/s 
     |████████████████████████████████| 753.4 MB 6.5 kB/s 
     |████████████████████████████████| 484 kB 45.1 MB/s 
     |████████████████████████████████| 73 kB 1.8 MB/s 
     |████████████████████████████████| 79 kB 6.9 MB/s 
     |████████████████████████████████| 449 kB 41.7 MB/s 
     |████████████████████████████████| 4.0 MB 42.0 MB/s 
     |████████████████████████████████| 126 kB 46.2 MB/s 
     |████████████████████████████████| 9.0 MB 24.4 MB/s 
     |████████████████████████████████| 71 kB 7.6 MB/s 
     |████████████████████████████████| 3.6 MB 31.3 MB/s 
     |█████████████████

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms

import syft as sy
hook = sy.TorchHook(torch)

In [3]:
# create the VirtualWorkers
bob = sy.VirtualWorker(hook, id="bob")  # define remote worker bob
alice = sy.VirtualWorker(hook, id="alice")  # define remote worker alice

In [4]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 10
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [6]:
# Data loading and sending to workers
federated_train_loader = sy.FederatedDataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate((bob, alice)), # distribute the dataset across all the workers, it's now a FederatedDataset
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize((0.1307,), (0.3081,))
    ])), batch_size=args.test_batch_size, shuffle=True, **kwargs)

In [7]:
# CNN specification
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [8]:
# Define the train and test functions
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader):     # a distributed dataset
        model.send(data.location)   # send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # get the model back
        if batch_idx % args.log_interval == 0:
            loss = loss.get()   # get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(train_loader) * args.batch_size, #batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True)   # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [9]:
# Launch the training
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr)   # TODO momentum is not supported at the moment

for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/60032 (0%)]	Loss: 2.311097
Train Epoch: 1 [640/60032 (1%)]	Loss: 2.279700
Train Epoch: 1 [1280/60032 (2%)]	Loss: 2.266318
Train Epoch: 1 [1920/60032 (3%)]	Loss: 2.214711
Train Epoch: 1 [2560/60032 (4%)]	Loss: 2.167513
Train Epoch: 1 [3200/60032 (5%)]	Loss: 2.109019
Train Epoch: 1 [3840/60032 (6%)]	Loss: 2.089679
Train Epoch: 1 [4480/60032 (7%)]	Loss: 2.024196
Train Epoch: 1 [5120/60032 (9%)]	Loss: 1.832399
Train Epoch: 1 [5760/60032 (10%)]	Loss: 1.634013
Train Epoch: 1 [6400/60032 (11%)]	Loss: 1.387239
Train Epoch: 1 [7040/60032 (12%)]	Loss: 1.220130
Train Epoch: 1 [7680/60032 (13%)]	Loss: 1.058031
Train Epoch: 1 [8320/60032 (14%)]	Loss: 0.805720
Train Epoch: 1 [8960/60032 (15%)]	Loss: 0.795623
Train Epoch: 1 [9600/60032 (16%)]	Loss: 0.646950
Train Epoch: 1 [10240/60032 (17%)]	Loss: 0.485977
Train Epoch: 1 [10880/60032 (18%)]	Loss: 0.536385
Train Epoch: 1 [11520/60032 (19%)]	Loss: 0.554034
Train Epoch: 1 [12160/60032 (20%)]	Loss: 0.571430
Train Epoch: 1 [12800/60032 (